In [7]:
import pandas as pd
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

#Load dataset
df = pd.read_csv('D:\Files\msg_class\question_list.csv')

#Mengonversi dataset menjadi tipe string dan menghapus tanda kutip dan spasi
df['message'] = df['Question List'].astype(str).str.replace('"', '').str.strip()

#Fungsi untuk preprocssing dataset
def clean_text(text):
    text = text.lower()                                                                                 #Mengubah semua teks menjadi huruf kecil (misal: "STB" dan "stb" dianggap sama)
    text = re.sub(r'\d+', '', text)                                                                     #Menghapus semua angka dari teks
    text = text.translate(str.maketrans('', '', string.punctuation))                                    #Menghapus tanda baca seperti titik, koma, tanda tanya, dll.
    return text.strip()                                                                                 #Menghapus spasi di awal dan akhir kalimat

df['clean_message'] = df['message'].apply(clean_text)

#Fungsi untuk pelabelan dataset berdasarkan kata kunci
def label_message(msg):
    problem_keywords = ['mati', 'putus', 'gangguan', 'tidak bisa', 'lemot', 'error', 'masalah', 'trouble', 'lambat', 'disconnect']
    request_keywords = ['relokasi', 'terminasi', 'ubah', 'ganti', 'mohon', 'minta', 'tolong', 'reset', 'pasang', 'instalasi', 'butuh', 'ingin', 'cek area', 'coverage', 'add on', 'datang teknisi']
    info_keywords = ['berapa', 'info', 'informasi', 'kapan', 'status', 'paket', 'harga', 'bayar', 'tagihan', 'promo']

    if any(kw in msg for kw in problem_keywords):
        return 'Problem'
    elif any(kw in msg for kw in request_keywords):
        return 'Request'
    elif any(kw in msg for kw in info_keywords):
        return 'Information'
    else:
        return 'Information'

df['category'] = df['clean_message'].apply(label_message)

#Split data menjadi data training (80% dari total data) dan data testing (20% dari total data)
X = df['clean_message']
y = df['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

train_df = pd.DataFrame({'message': X_train, 'category': y_train})                                      #Membuat dataframe dari data training, berisi pesan dan kategorinya
test_df = pd.DataFrame({'message': X_test, 'category': y_test})                                         #Membuat dataframe dari data testing, berisi pesan dan kategorinya

train_df.to_csv('dataset_training.csv', index=False)                                                    #Menyimpan data training ke file CSV
test_df.to_csv('dataset_testing.csv', index=False)                                                      #Menyimpan data testing ke file CSV

#Buat dan latih model klasifikasi Naive Bayes
model = Pipeline([
    ('tfidf', TfidfVectorizer()),                                                                       #Mengubah teks menjadi matriks angka
    ('clf', MultinomialNB())                                                                            #Model klasifikasi menggunakan algoritma Multinomial Naive Bayes
])
model.fit(X_train, y_train)                                                                             #Melatih model menggunakan data training

#Menguji model klasifikasi untuk memprediksi kategori dari data testing 
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)                                                               #Menghitung akurasi model klasifikasi
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)                        #Menghitung presisi model klasifikasi
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)                              #Menghitung recall model klasifikasi

#Menyimpan data hasil pengujian model ke dataframe baru
df_test = X_test.to_frame()
df_test['actual'] = y_test.values                                                                       #Menambahkan kolom 'actual' yang berisi label sebenarnya dari data testing
df_test['predicted'] = y_pred                                                                           #Menambahkan kolom 'predicted' yang berisi hasil prediksi model terhadap data testing

df_test.to_csv("hasil_klasifikasi.csv", index=False)                                                    #Menyimpan hasil klasifikasi ke file CSV

#Menampilkan 5 contoh pesan dari tiap kategori prediksi
for category in ['Information', 'Request', 'Problem']:
    print(f"\n=== Contoh prediksi kategori: {category} ===")
    print(df_test[df_test['predicted'] == category][['clean_message', 'predicted', 'actual']].head(5).to_string(index=False))

#Menampilkan hasil dari perhitungan matriks
print("\n=== Evaluasi Model ===")
print(f"Akurasi: {accuracy:.2f}")
print(f"Presisi: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))



=== Contoh prediksi kategori: Information ===
                                                                                     clean_message   predicted      actual
jika banyak perangkat yang terhubung pada modem apakah akan berpengaruh pada speed yang didapatkan Information Information
                                                                                           ada hbo Information Information
                                                                                 internet down nih Information Information
                                                                   apakah ada promo khusus untukku Information Information
                                                                                    masalah teknis Information     Problem

=== Contoh prediksi kategori: Request ===
                    clean_message predicted  actual
                  mau ganti paket   Request Request
apa bisa ganti kepemilikan biznet   Request Request
                